In [4]:
# %% Import Libraries
import json
import os
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import traceback
import re
from scipy import stats  # Para análisis estadístico

# %% Configuration
pio.templates.default = "plotly_white"
CTI = 0.2  # Coordination Time Interval
DPI = 300  # Alta resolución para imágenes

# --- Define Input Files and Output Directory ---
base_dir = "/Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/processed/"
output_dir_comparison = "/Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/results/comparison_plots_thesis"

# Crear directorio de salida
os.makedirs(output_dir_comparison, exist_ok=True)

# Fuentes de datos
data_sources = [
    {"name": "Automation", "file": os.path.join(base_dir, "independent_relay_pairs_automation.json"), "time_key": "Time_out"},
    {"name": "Optimization 1", "file": os.path.join(base_dir, "independent_relay_pairs_optimization.json"), "time_key": "Time_out"},
    {"name": "Optimization 2", "file": os.path.join(base_dir, "independent_relay_pairs_second_optimization.json"), "time_key": "Time_out"},
    {"name": "Transformer", "file": os.path.join(base_dir, "independent_relay_pairs_transformer.json"), "time_key": "new_timeout"}
]

# --- Plotting Style (Optimizado para tesis, más delgado) ---
FIG_WIDTH = 5.0  # Pulgadas (~12.7 cm) para hoja carta, más delgado
FIG_HEIGHT = 4.5  # Pulgadas (~11.4 cm)
TITLE_FONT_SIZE = 14  # Reducido para ajustar al espacio
AXIS_LABEL_FONT_SIZE = 12
TICK_FONT_SIZE = 9  # Reducido para legibilidad en figura estrecha
LEGEND_FONT_SIZE = 9
COLORS = {'Automation': '#1f77b4', 'Optimization 1': '#ff7f0e', 'Optimization 2': '#2ca02c', 'Transformer': '#d62728'}

# %% Reusable Analysis Function
def analyze_data_source(file_path, source_name, time_key='Time_out', cti=CTI):
    """
    Analiza un archivo JSON de pares de relés y calcula métricas por escenario.
    """
    scenario_results = {}
    print(f"\n--- Processing: {source_name} ---")
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            relay_pairs_data = json.load(f)
        if not isinstance(relay_pairs_data, list):
            raise TypeError(f"Error: File {file_path} does not contain a JSON list.")

        total_pairs_read = 0
        skipped_pairs_count = 0
        for pair_entry in relay_pairs_data:
            total_pairs_read += 1
            if not isinstance(pair_entry, dict):
                skipped_pairs_count += 1
                continue
            scenario_id = pair_entry.get("scenario_id")
            if not scenario_id:
                skipped_pairs_count += 1
                continue
            if scenario_id not in scenario_results:
                scenario_results[scenario_id] = {'tmt': 0.0, 'coordinated': 0, 'uncoordinated': 0, 'total_valid': 0}
            main_relay_info = pair_entry.get('main_relay')
            backup_relay_info = pair_entry.get('backup_relay')
            if not isinstance(main_relay_info, dict) or not isinstance(backup_relay_info, dict):
                skipped_pairs_count += 1
                continue
            main_time = main_relay_info.get(time_key)
            backup_time = backup_relay_info.get(time_key)
            try:
                if main_time is None or backup_time is None:
                    raise TypeError("Missing time value")
                main_time_f = float(main_time)
                backup_time_f = float(backup_time)
                if main_time_f < 0 or backup_time_f < 0:
                    raise ValueError("Negative time value")
            except (ValueError, TypeError):
                skipped_pairs_count += 1
                continue
            delta_t = backup_time_f - main_time_f - cti
            mt = (delta_t - abs(delta_t)) / 2
            scenario_results[scenario_id]['tmt'] += mt
            scenario_results[scenario_id]['total_valid'] += 1
            if delta_t >= 0:
                scenario_results[scenario_id]['coordinated'] += 1
            else:
                scenario_results[scenario_id]['uncoordinated'] += 1

        print(f"Finished processing {source_name}. Total pairs: {total_pairs_read}, Skipped: {skipped_pairs_count}")
        return scenario_results
    except Exception as e:
        print(f"CRITICAL ERROR ({source_name}): {e}")
        return {}

# %% Plotting Function
def plot_comparison(df, metric, title, y_label, output_prefix, sorted_scenario_names, show_plots=True):
    """
    Genera un gráfico de barras vertical más delgado para comparar métricas entre métodos y escenarios.
    """
    fig = px.bar(
        df,
        y='Escenario', x=metric, color='Metodo', barmode='group',
        title=title, labels={metric: y_label, 'Metodo': 'Método'},
        hover_data=['Pares Coordinados', 'Pares Descoordinados', 'Total Pares Válidos', 'Porcentaje Coordinación'],
        text=metric, orientation='h'
    )
    fig.update_traces(
        texttemplate='%{text:.4f}' if metric == 'TMT' else '%{text:.1f}%',
        textposition='outside', marker=dict(line=dict(color='black', width=0.5))
    )
    fig.update_layout(
        yaxis=dict(
            title='Escenarios',
            categoryorder='array',
            categoryarray=sorted_scenario_names,
            gridcolor='LightGray',
            tickfont=dict(size=TICK_FONT_SIZE)
        ),
        xaxis=dict(
            title=y_label,
            gridcolor='LightGray',
            zerolinecolor='black',
            zerolinewidth=1,
            tickfont=dict(size=TICK_FONT_SIZE)
        ),
        legend_title_text='Método',
        font=dict(family="Arial", size=TICK_FONT_SIZE),
        title_font_size=TITLE_FONT_SIZE,
        xaxis_title_font_size=AXIS_LABEL_FONT_SIZE,
        yaxis_title_font_size=AXIS_LABEL_FONT_SIZE,
        legend_font_size=LEGEND_FONT_SIZE,
        height=FIG_HEIGHT * DPI,
        width=FIG_WIDTH * DPI,
        margin=dict(l=80, r=30, t=60, b=40),  # Márgenes más compactos
        bargap=0.15,  # Reducir espacio entre barras
        colorway=[COLORS[m] for m in df['Metodo'].unique() if m in COLORS]
    )
    if metric == 'Porcentaje Coordinación':
        fig.update_xaxes(range=[0, 100])
    base_filename = os.path.join(output_dir_comparison, output_prefix)
    fig.write_html(f"{base_filename}.html")
    for fmt in ['png', 'svg', 'pdf']:
        try:
            fig.write_image(f"{base_filename}.{fmt}", format=fmt, scale=2 if fmt == 'png' else 1)
            print(f"    Saved: {base_filename}.{fmt}")
        except Exception as e:
            print(f"    Error saving {base_filename}.{fmt}: {e}")
    if show_plots:
        fig.show()

# %% Statistical Analysis Function
def compute_statistics(df):
    """
    Calcula estadísticas detalladas para TMT y Porcentaje de Coordinación por método.
    """
    print("\n--- Análisis Estadístico Detallado ---")
    stats_summary = []
    for metodo in df['Metodo'].unique():
        df_metodo = df[df['Metodo'] == metodo]
        for metric in ['TMT', 'Porcentaje Coordinación']:
            data = df_metodo[metric].dropna()
            if data.empty:
                continue
            stat = {
                'Método': metodo,
                'Métrica': metric,
                'Media': data.mean(),
                'Mediana': data.median(),
                'Desviación Estándar': data.std(),
                'Mínimo': data.min(),
                'Máximo': data.max(),
                'Percentil 25': data.quantile(0.25),
                'Percentil 75': data.quantile(0.75)
            }
            stats_summary.append(stat)
    
    df_stats = pd.DataFrame(stats_summary)
    print("\nEstadísticas por Método y Métrica:")
    print(df_stats.to_string(index=False))
    
    # Comparaciones entre métodos (diferencias promedio)
    print("\nComparaciones entre Métodos (Diferencias Promedio en TMT):")
    methods = df['Metodo'].unique()
    for i in range(len(methods)):
        for j in range(i + 1, len(methods)):
            m1, m2 = methods[i], methods[j]
            tmt_diff = df[df['Metodo'] == m1]['TMT'].mean() - df[df['Metodo'] == m2]['TMT'].mean()
            coord_diff = df[df['Metodo'] == m1]['Porcentaje Coordinación'].mean() - df[df['Metodo'] == m2]['Porcentaje Coordinación'].mean()
            print(f"  {m1} vs {m2}: TMT Diff = {tmt_diff:.4f}, Coord % Diff = {coord_diff:.2f}%")
    
    # Guardar estadísticas
    stats_path = os.path.join(output_dir_comparison, "statistical_analysis.csv")
    df_stats.to_csv(stats_path, index=False, float_format='%.4f')
    print(f"\nEstadísticas guardadas en: {stats_path}")
    
    return df_stats

# %% Main Execution Logic
def main():
    print("========================================================")
    print("=== Relay Coordination Comparison Analysis Script ===")
    print("========================================================")

    all_results_list = []
    for source in data_sources:
        results = analyze_data_source(source["file"], source["name"], source["time_key"], CTI)
        if not results:
            continue
        for scenario, data in results.items():
            if data['total_valid'] > 0:
                coord_percentage = 100 * data['coordinated'] / data['total_valid']
            else:
                coord_percentage = 0
            all_results_list.append({
                'Metodo': source['name'], 'Escenario': scenario, 'TMT': data['tmt'],
                'Pares Coordinados': data['coordinated'], 'Pares Descoordinados': data['uncoordinated'],
                'Total Pares Válidos': data['total_valid'], 'Porcentaje Coordinación': coord_percentage
            })

    if not all_results_list:
        print("\nCRITICAL ERROR: No valid data processed. Exiting.")
        return

    df_combined = pd.DataFrame(all_results_list)
    try:
        df_combined['Scenario_Num'] = df_combined['Escenario'].str.extract(r'(\d+)$', expand=False).astype(int)
        df_combined_sorted = df_combined.sort_values(by=['Scenario_Num', 'Metodo']).reset_index(drop=True)
    except Exception:
        df_combined_sorted = df_combined.sort_values(by=['Escenario', 'Metodo']).reset_index(drop=True)

    # Generar gráficos
    sorted_scenario_names = df_combined_sorted['Escenario'].unique().tolist()
    plot_comparison(
        df_combined_sorted, 'TMT', 'Comparación de Penalización Total (TMT) por Escenario y Método',
        'TMT (Suma de Penalizaciones)', 'comparison_tmt_by_scenario_thesis', sorted_scenario_names
    )
    plot_comparison(
        df_combined_sorted, 'Porcentaje Coordinación', 'Comparación de Porcentaje de Coordinación por Escenario y Método',
        'Porcentaje de Coordinación (%)', 'comparison_coordination_by_scenario_thesis', sorted_scenario_names
    )

    # Tabla resumen
    print("\n--- Tabla Resumen Comparativa ---")
    df_pivot_tmt = df_combined_sorted.pivot_table(index='Escenario', columns='Metodo', values='TMT')
    df_pivot_coord = df_combined_sorted.pivot_table(index='Escenario', columns='Metodo', values='Porcentaje Coordinación')
    method_order = [src['name'] for src in data_sources if src['name'] in df_pivot_tmt.columns]
    df_pivot_tmt = df_pivot_tmt[method_order]
    df_pivot_coord = df_pivot_coord[method_order]
    df_pivot_tmt.columns = pd.MultiIndex.from_product([['TMT'], df_pivot_tmt.columns])
    df_pivot_coord.columns = pd.MultiIndex.from_product([['Coord %'], df_pivot_coord.columns])
    df_summary_pivot = pd.concat([df_pivot_tmt, df_pivot_coord], axis=1)
    scenario_order_map = {name: num for name, num in zip(df_combined_sorted['Escenario'], df_combined_sorted['Scenario_Num'])}
    df_summary_pivot = df_summary_pivot.sort_index(key=lambda x: x.map(scenario_order_map))
    pd.set_option('display.float_format', '{:.4f}'.format)
    print(df_summary_pivot)
    summary_table_path = os.path.join(output_dir_comparison, "comparison_summary_table.csv")
    df_summary_pivot.to_csv(summary_table_path, float_format='%.5f')
    print(f"Summary table saved to: {summary_table_path}")

    # Análisis estadístico
    compute_statistics(df_combined_sorted)

    print("\n=== Analysis Finished ===")

# %% Script Entry Point
if __name__ == "__main__":
    main()

=== Relay Coordination Comparison Analysis Script ===

--- Processing: Automation ---
Finished processing Automation. Total pairs: 6800, Skipped: 0

--- Processing: Optimization 1 ---
Finished processing Optimization 1. Total pairs: 6800, Skipped: 0

--- Processing: Optimization 2 ---
Finished processing Optimization 2. Total pairs: 6800, Skipped: 0

--- Processing: Transformer ---
Finished processing Transformer. Total pairs: 6800, Skipped: 0
    Saved: /Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/results/comparison_plots_thesis/comparison_tmt_by_scenario_thesis.png
    Saved: /Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/results/comparison_plots_thesis/comparison_tmt_by_scenario_thesis.svg
    Saved: /Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/results/comparison_plots_thesis/comparison_tmt_by_scenario_thesis.pdf


    Saved: /Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/results/comparison_plots_thesis/comparison_coordination_by_scenario_thesis.png
    Saved: /Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/results/comparison_plots_thesis/comparison_coordination_by_scenario_thesis.svg
    Saved: /Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/results/comparison_plots_thesis/comparison_coordination_by_scenario_thesis.pdf



--- Tabla Resumen Comparativa ---
                   TMT                                              Coord %  \
Metodo      Automation Optimization 1 Optimization 2 Transformer Automation   
Escenario                                                                     
scenario_1    -15.8543        -0.7393        -0.7393     -1.0214    12.0000   
scenario_2    -18.9624       -17.0231        -2.9126     -2.5852    12.0000   
scenario_3    -17.8215       -17.7316        -2.3719     -1.8231    13.0000   
scenario_4    -15.2305        -0.5175        -0.5175     -0.5956    13.0000   
scenario_5    -24.9475       -16.6551        -1.8020     -0.9599    13.0000   
...                ...            ...            ...         ...        ...   
scenario_64   -15.3634        -1.8093        -1.8093     -1.3843    11.0000   
scenario_65   -14.8151        -0.8416        -0.8416     -1.7497    15.0000   
scenario_66   -14.9448        -0.2330        -0.2330     -0.4726    14.0000   
scenario_67   -14

In [3]:
# %% Import Libraries
import json
import os
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import traceback
import re
from scipy import stats  # Para análisis estadístico

# %% Configuration
pio.templates.default = "plotly_white"
CTI = 0.2  # Coordination Time Interval
DPI = 300  # Alta resolución para imágenes

# --- Define Input Files and Output Directory ---
base_dir = "/Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/processed/"
output_dir_comparison = "/Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/results/comparison_plots_thesis"

# Crear directorio de salida
os.makedirs(output_dir_comparison, exist_ok=True)

# Fuentes de datos
data_sources = [
    {"name": "Automation", "file": os.path.join(base_dir, "independent_relay_pairs_automation.json"), "time_key": "Time_out"},
    {"name": "Optimization 1", "file": os.path.join(base_dir, "independent_relay_pairs_optimization.json"), "time_key": "Time_out"},
    {"name": "Optimization 2", "file": os.path.join(base_dir, "independent_relay_pairs_second_optimization.json"), "time_key": "Time_out"},
    {"name": "Transformer", "file": os.path.join(base_dir, "independent_relay_pairs_transformer.json"), "time_key": "new_timeout"}
]

# --- Plotting Style (Optimizado para tesis) ---
FIG_WIDTH = 6.5  # Pulgadas (~16.5 cm) para hoja carta
FIG_HEIGHT = 4.5  # Pulgadas (~11.4 cm)
TITLE_FONT_SIZE = 16
AXIS_LABEL_FONT_SIZE = 12
TICK_FONT_SIZE = 10
LEGEND_FONT_SIZE = 10
COLORS = {'Automation': '#1f77b4', 'Optimization 1': '#ff7f0e', 'Optimization 2': '#2ca02c', 'Transformer': '#d62728'}

# %% Reusable Analysis Function
def analyze_data_source(file_path, source_name, time_key='Time_out', cti=CTI):
    """
    Analiza un archivo JSON de pares de relés y calcula métricas por escenario.
    """
    scenario_results = {}
    print(f"\n--- Processing: {source_name} ---")
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            relay_pairs_data = json.load(f)
        if not isinstance(relay_pairs_data, list):
            raise TypeError(f"Error: File {file_path} does not contain a JSON list.")

        total_pairs_read = 0
        skipped_pairs_count = 0
        for pair_entry in relay_pairs_data:
            total_pairs_read += 1
            if not isinstance(pair_entry, dict):
                skipped_pairs_count += 1
                continue
            scenario_id = pair_entry.get("scenario_id")
            if not scenario_id:
                skipped_pairs_count += 1
                continue
            if scenario_id not in scenario_results:
                scenario_results[scenario_id] = {'tmt': 0.0, 'coordinated': 0, 'uncoordinated': 0, 'total_valid': 0}
            main_relay_info = pair_entry.get('main_relay')
            backup_relay_info = pair_entry.get('backup_relay')
            if not isinstance(main_relay_info, dict) or not isinstance(backup_relay_info, dict):
                skipped_pairs_count += 1
                continue
            main_time = main_relay_info.get(time_key)
            backup_time = backup_relay_info.get(time_key)
            try:
                if main_time is None or backup_time is None:
                    raise TypeError("Missing time value")
                main_time_f = float(main_time)
                backup_time_f = float(backup_time)
                if main_time_f < 0 or backup_time_f < 0:
                    raise ValueError("Negative time value")
            except (ValueError, TypeError):
                skipped_pairs_count += 1
                continue
            delta_t = backup_time_f - main_time_f - cti
            mt = (delta_t - abs(delta_t)) / 2
            scenario_results[scenario_id]['tmt'] += mt
            scenario_results[scenario_id]['total_valid'] += 1
            if delta_t >= 0:
                scenario_results[scenario_id]['coordinated'] += 1
            else:
                scenario_results[scenario_id]['uncoordinated'] += 1

        print(f"Finished processing {source_name}. Total pairs: {total_pairs_read}, Skipped: {skipped_pairs_count}")
        return scenario_results
    except Exception as e:
        print(f"CRITICAL ERROR ({source_name}): {e}")
        return {}

# %% Plotting Function
def plot_comparison(df, metric, title, y_label, output_prefix, sorted_scenario_names, show_plots=True):
    """
    Genera un gráfico de barras vertical para comparar métricas entre métodos y escenarios.
    """
    fig = px.bar(
        df,
        y='Escenario', x=metric, color='Metodo', barmode='group',
        title=title, labels={metric: y_label, 'Metodo': 'Método'},
        hover_data=['Pares Coordinados', 'Pares Descoordinados', 'Total Pares Válidos', 'Porcentaje Coordinación'],
        text=metric, orientation='h'
    )
    fig.update_traces(
        texttemplate='%{text:.4f}' if metric == 'TMT' else '%{text:.1f}%',
        textposition='outside', marker=dict(line=dict(color='black', width=0.5))
    )
    fig.update_layout(
        yaxis=dict(
            title='Escenarios',
            categoryorder='array',
            categoryarray=sorted_scenario_names,
            gridcolor='LightGray'
        ),
        xaxis=dict(
            title=y_label,
            gridcolor='LightGray',
            zerolinecolor='black',
            zerolinewidth=1
        ),
        legend_title_text='Método',
        font=dict(family="Arial", size=TICK_FONT_SIZE),
        title_font_size=TITLE_FONT_SIZE,
        xaxis_title_font_size=AXIS_LABEL_FONT_SIZE,
        yaxis_title_font_size=AXIS_LABEL_FONT_SIZE,
        xaxis_tickfont_size=TICK_FONT_SIZE,
        yaxis_tickfont_size=TICK_FONT_SIZE,
        legend_font_size=LEGEND_FONT_SIZE,
        height=FIG_HEIGHT * DPI,
        width=FIG_WIDTH * DPI,
        margin=dict(l=100, r=50, t=80, b=50),
        colorway=[COLORS[m] for m in df['Metodo'].unique() if m in COLORS]
    )
    if metric == 'Porcentaje Coordinación':
        fig.update_xaxes(range=[0, 100])
    base_filename = os.path.join(output_dir_comparison, output_prefix)
    fig.write_html(f"{base_filename}.html")
    for fmt in ['png', 'svg', 'pdf']:
        try:
            fig.write_image(f"{base_filename}.{fmt}", format=fmt, scale=2 if fmt == 'png' else 1)
            print(f"    Saved: {base_filename}.{fmt}")
        except Exception as e:
            print(f"    Error saving {base_filename}.{fmt}: {e}")
    if show_plots:
        fig.show()

# %% Statistical Analysis Function
def compute_statistics(df):
    """
    Calcula estadísticas detalladas para TMT y Porcentaje de Coordinación por método.
    """
    print("\n--- Análisis Estadístico Detallado ---")
    stats_summary = []
    for metodo in df['Metodo'].unique():
        df_metodo = df[df['Metodo'] == metodo]
        for metric in ['TMT', 'Porcentaje Coordinación']:
            data = df_metodo[metric].dropna()
            if data.empty:
                continue
            stat = {
                'Método': metodo,
                'Métrica': metric,
                'Media': data.mean(),
                'Mediana': data.median(),
                'Desviación Estándar': data.std(),
                'Mínimo': data.min(),
                'Máximo': data.max(),
                'Percentil 25': data.quantile(0.25),
                'Percentil 75': data.quantile(0.75)
            }
            stats_summary.append(stat)
    
    df_stats = pd.DataFrame(stats_summary)
    print("\nEstadísticas por Método y Métrica:")
    print(df_stats.to_string(index=False))
    
    # Comparaciones entre métodos (diferencias promedio)
    print("\nComparaciones entre Métodos (Diferencias Promedio en TMT):")
    methods = df['Metodo'].unique()
    for i in range(len(methods)):
        for j in range(i + 1, len(methods)):
            m1, m2 = methods[i], methods[j]
            tmt_diff = df[df['Metodo'] == m1]['TMT'].mean() - df[df['Metodo'] == m2]['TMT'].mean()
            coord_diff = df[df['Metodo'] == m1]['Porcentaje Coordinación'].mean() - df[df['Metodo'] == m2]['Porcentaje Coordinación'].mean()
            print(f"  {m1} vs {m2}: TMT Diff = {tmt_diff:.4f}, Coord % Diff = {coord_diff:.2f}%")
    
    # Guardar estadísticas
    stats_path = os.path.join(output_dir_comparison, "statistical_analysis.csv")
    df_stats.to_csv(stats_path, index=False, float_format='%.4f')
    print(f"\nEstadísticas guardadas en: {stats_path}")
    
    return df_stats

# %% Main Execution Logic
def main():
    print("========================================================")
    print("=== Relay Coordination Comparison Analysis Script ===")
    print("========================================================")

    all_results_list = []
    for source in data_sources:
        results = analyze_data_source(source["file"], source["name"], source["time_key"], CTI)
        if not results:
            continue
        for scenario, data in results.items():
            if data['total_valid'] > 0:
                coord_percentage = 100 * data['coordinated'] / data['total_valid']
            else:
                coord_percentage = 0
            all_results_list.append({
                'Metodo': source['name'], 'Escenario': scenario, 'TMT': data['tmt'],
                'Pares Coordinados': data['coordinated'], 'Pares Descoordinados': data['uncoordinated'],
                'Total Pares Válidos': data['total_valid'], 'Porcentaje Coordinación': coord_percentage
            })

    if not all_results_list:
        print("\nCRITICAL ERROR: No valid data processed. Exiting.")
        return

    df_combined = pd.DataFrame(all_results_list)
    try:
        df_combined['Scenario_Num'] = df_combined['Escenario'].str.extract(r'(\d+)$', expand=False).astype(int)
        df_combined_sorted = df_combined.sort_values(by=['Scenario_Num', 'Metodo']).reset_index(drop=True)
    except Exception:
        df_combined_sorted = df_combined.sort_values(by=['Escenario', 'Metodo']).reset_index(drop=True)

    # Generar gráficos
    sorted_scenario_names = df_combined_sorted['Escenario'].unique().tolist()
    plot_comparison(
        df_combined_sorted, 'TMT', 'Comparación de Penalización Total (TMT) por Escenario y Método',
        'TMT (Suma de Penalizaciones)', 'comparison_tmt_by_scenario_thesis', sorted_scenario_names
    )
    plot_comparison(
        df_combined_sorted, 'Porcentaje Coordinación', 'Comparación de Porcentaje de Coordinación por Escenario y Método',
        'Porcentaje de Coordinación (%)', 'comparison_coordination_by_scenario_thesis', sorted_scenario_names
    )

    # Tabla resumen
    print("\n--- Tabla Resumen Comparativa ---")
    df_pivot_tmt = df_combined_sorted.pivot_table(index='Escenario', columns='Metodo', values='TMT')
    df_pivot_coord = df_combined_sorted.pivot_table(index='Escenario', columns='Metodo', values='Porcentaje Coordinación')
    method_order = [src['name'] for src in data_sources if src['name'] in df_pivot_tmt.columns]
    df_pivot_tmt = df_pivot_tmt[method_order]
    df_pivot_coord = df_pivot_coord[method_order]
    df_pivot_tmt.columns = pd.MultiIndex.from_product([['TMT'], df_pivot_tmt.columns])
    df_pivot_coord.columns = pd.MultiIndex.from_product([['Coord %'], df_pivot_coord.columns])
    df_summary_pivot = pd.concat([df_pivot_tmt, df_pivot_coord], axis=1)
    scenario_order_map = {name: num for name, num in zip(df_combined_sorted['Escenario'], df_combined_sorted['Scenario_Num'])}
    df_summary_pivot = df_summary_pivot.sort_index(key=lambda x: x.map(scenario_order_map))
    pd.set_option('display.float_format', '{:.4f}'.format)
    print(df_summary_pivot)
    summary_table_path = os.path.join(output_dir_comparison, "comparison_summary_table.csv")
    df_summary_pivot.to_csv(summary_table_path, float_format='%.5f')
    print(f"Summary table saved to: {summary_table_path}")

    # Análisis estadístico
    compute_statistics(df_combined_sorted)

    print("\n=== Analysis Finished ===")

# %% Script Entry Point
if __name__ == "__main__":
    main()

=== Relay Coordination Comparison Analysis Script ===

--- Processing: Automation ---
Finished processing Automation. Total pairs: 6800, Skipped: 0

--- Processing: Optimization 1 ---
Finished processing Optimization 1. Total pairs: 6800, Skipped: 0

--- Processing: Optimization 2 ---
Finished processing Optimization 2. Total pairs: 6800, Skipped: 0

--- Processing: Transformer ---
Finished processing Transformer. Total pairs: 6800, Skipped: 0
    Saved: /Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/results/comparison_plots_thesis/comparison_tmt_by_scenario_thesis.png
    Saved: /Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/results/comparison_plots_thesis/comparison_tmt_by_scenario_thesis.svg
    Saved: /Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/results/comparison_plots_thesis/comparison_tmt_by_scenario_thesis.pdf


    Saved: /Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/results/comparison_plots_thesis/comparison_coordination_by_scenario_thesis.png
    Saved: /Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/results/comparison_plots_thesis/comparison_coordination_by_scenario_thesis.svg
    Saved: /Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/results/comparison_plots_thesis/comparison_coordination_by_scenario_thesis.pdf



--- Tabla Resumen Comparativa ---
                   TMT                                              Coord %  \
Metodo      Automation Optimization 1 Optimization 2 Transformer Automation   
Escenario                                                                     
scenario_1    -15.8543        -0.7393        -0.7393     -1.0214    12.0000   
scenario_2    -18.9624       -17.0231        -2.9126     -2.5852    12.0000   
scenario_3    -17.8215       -17.7316        -2.3719     -1.8231    13.0000   
scenario_4    -15.2305        -0.5175        -0.5175     -0.5956    13.0000   
scenario_5    -24.9475       -16.6551        -1.8020     -0.9599    13.0000   
...                ...            ...            ...         ...        ...   
scenario_64   -15.3634        -1.8093        -1.8093     -1.3843    11.0000   
scenario_65   -14.8151        -0.8416        -0.8416     -1.7497    15.0000   
scenario_66   -14.9448        -0.2330        -0.2330     -0.4726    14.0000   
scenario_67   -14